In [ ]:
from openai import OpenAI
from docx import Document
import pandas as pd
import os
import json
import re
import time

# -----------------------------
# LLM Client Setup
# -----------------------------
client = OpenAI(
    base_url="http://10.173.91.239:1234/v1",
    api_key="lm-studio"
)

model_name = "google/gemma-3-4b"

# -----------------------------
# File Paths
# -----------------------------
INPUT_FILE = "750-tweets-without answer.xlsx"
OUTPUT_FILE = "Gemma3_old_prompt1.xlsx"


# -----------------------------
# JSON Schema for classification
# -----------------------------
classification_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "AgeismClassification",
        "schema": {
            "type": "object",
            "properties": {
                "classification": {
                    "type": "string",
                    "enum": ["Not relevant", "Ageist", "Age-positive"]
                }
            },
            "required": ["classification"]
        },
    }
}


# -----------------------------
# Load input Excel file
# -----------------------------
df = pd.read_excel(INPUT_FILE, dtype=str)
df["classification"] = ""   # empty column to fill later

# Load keyword list
keywords_doc = Document("Keywords list_final.docx")
keywords_text = "\n".join([p.text for p in keywords_doc.paragraphs])

# -----------------------------
# Loop through tweets
# -----------------------------
for i, row in df.iterrows():
    start_time = time.time()
    text = row["text"]  # make sure your tweet text column is called "text"

    prompt_text = f"""
    Our goal is to create 3 lists of tweets.  Please divide all the tweets into one of 3 categories: 
1)	 ageist tweets; 2) age-positive tweets; and 3) not relevant tweets. 

Our definition of “ageist” tweets are tweets that are related to older persons and show an unfavorable portrayal and/or treatment of older persons by societal institutions and those affiliated with them. 

Our definition of “age-positive” tweets are those that are related to older persons and show the favorable portrayal and/or treatment of older persons by societal institutions and those affiliated with them.

Our definition of “not relevant” tweets are those that are not related to older persons and/ or are related to older persons but describes them in a neutral way that is not ageist or age-positive.

To create these 3 lists, first identify all tweets that describe an older person or older people (someone in their later decades) using Keyword List A from the attached document. Consider the context carefully. 


When classifying a tweet as related to older persons, please note the following rules. Please do not consider a tweet related to an older person when: 
(1) tweets describe old objects (e.g., “old car,” “old house”) unless the term refers to an old body part (e.g., “old knees” or “old brain”). 
(2) “old,” refers to “old + testament” or old refers to something that is not an older person, like "the old you" "the old we" "old time" "old adage" or "old outhouse"  
(3) “senior(s)” refers to high school or college seniors. 
(4) “pop” refers to soda, a verb, music or could refer to a younger person (e.g., “my pop” could mean a young “dad”). 
(5) For “elder” refers to Elder Scrolls (the video game). 
(6) For “granny,” refers to granny smith which is an apple. 
(7) For “antique” and “ancient,” describes objects rather than people. 
(8) For “nursing home,” “assisted living,” or “care home,” does not refer to older persons.
If tweet includes a positive sentiment about an older person, such as when a tweet mentions that they miss an older person or they feel sad that an older person is no longer around, classify tweet as “age positive.”
Once you have identified the tweets describing older persons, classify them into 3 subsets: (1) ageist tweets using Keyword List B and the definition above, and (2) age-positive tweets using Keyword List C and the corresponding definition above or 3) not relevant tweets. A tweet cannot be both ageist and age-positive.


The final dataset should include the following columns for each tweet: aging-related (yes/no), ageist (yes/no), age-positive (yes/no), not relevant (yes/ no) and reasoning/notes explaining why the classification was made (e.g., “uses stereotype of frailty” or “praises older adults for resilience”).

Also, please see the correct classifications of the following 8 tweets that could assist as you decide how to classify tweets: 
1. "Everybody grumpy today"
   - Keywords: "grumpy" (negative age stereotype)
   - Classification: Not relevant to ageist or age-positive list (as it does not specifically refer to older persons)
 
2. "Y'all clamoring for the days of the Old Testament y'all would be slaves in those days"
   - Keywords: "Old Testament" (irrelevant as it refers to the Bible, not older people)
   - Classification: Not relevant to ageist or age-positive list
 
3. "I lost my granny early in life but I cherish the moments I had with her forever"
   - Keywords: "granny"
   - Classification: Age-positive (demonstrates value and positive memory of an older person)
  
4. "WHAT?! You missed Grandparents Storytime last week? Guess what? You have a chance to catch it again on Saturday, March 21, at 10:30 a.m. Miss Tracey will sing songs, have craft time and share wonderful stories for…"
   - Keywords: "Grandparents"
   - Classification: Age-positive (promotes engagement and positive role of grandparents)
 
5. "@captaintommoore has died from #COVID19. The 100-year-old helped raise almost £33m ($57M Cdn.) for the NHS. He was knighted last year. The Army veteran won the hearts of millions by walking 100 laps in his garden before his 100th birthday last April. #Hero #Positivity #Smile"
   - Keywords: "100-year-old", "Hero", "Positivity"
   - Classification: Age-positive (highlights achievements and positive contributions of an older person)
 
6. "@Hichestan7 @SecPompeo @mikepompeo Still drinking from Trump’s septic tank? Its not good for you. The lies & bullshit eat away at your brain, accelerating dementia."
   - Keywords: "dementia" (negative age stereotype)
   - Classification: Ageist (uses dementia derogatorily)
 
7. "My old man bladder is so thankful! Not a lot more stressful than worrying if you can hold it the whole movie or not!"
   - Keywords: "old man bladder" (negative age reference)
   - Classification: Ageist (implies bladder issues due to old age negatively)
 
8. "Hawk Nation tune-in Sun, 4/19 on #Knightline as Coach Derek Smith (@chowan6464) discusses the recent UCF verbal commitment by Hawks speedy senior WR @davismallinger1 Live from 10-11:00 am with coach’s segment airing 10:20 am. ESPN Orlando 580 AM Hosted by @AP_Knightline https://t.co/yVmMX2jTyn"
   - Keywords: "senior"
   - Classification: Not relevant to ageist or age-positive list (senior refers to high school senior, not older person)

In addition to the rules listed above, please apply the following clarification rules when determining whether a tweet is aging-related and how it should be classified:
Tweets that mention dementia, Alzheimer’s disease, memory loss, cognitive decline, or similar conditions should always be classified as ageist, even if the tone appears sympathetic. This is because dementia is a negative stereotype associated with aging and reinforces age-based decline.
Tweets that portray an older adult in a negative, mocking, complaining, or burdensome way (such as criticizing their cooking, speech, abilities, slowness, confusion, or behavior) should be classified as ageist, even if the tweet is casual or humorous. For example, “my grandma’s cooking smells terrible” or “grandpa won’t stop repeating himself” are ageist.
Tweets that express positive affection, respect, gratitude, pride, love, or admiration for an older adult should be classified as age-positive, including when someone says they miss or cherish an older relative who has passed away. Emotional remembrance counts as age-positive.
Tweets that simply mention an older person without expressing a positive or negative judgment are considered aging-related but neutral (coded as “not relevant” in the final 3-way grouping).
The keyword “old people / old ppl / old folks” does not automatically refer to older adults and should only be coded as aging-related if the context makes it clear that the speaker is referring to elderly individuals rather than adults in general.
The keyword “elder” is not the same as “elderly” and should not be coded as aging-related unless the context clearly refers to an older person. For example, “elder sibling,” “church elder,” or “ancestral elder” should not be counted.
The word “retired” should only be coded as aging-related when it describes a person retiring due to age, not when it is used jokingly (e.g., “I’m retired from dating”).
The term “pops” is not automatically an older adult and may refer to a younger father unless clearly stated otherwise. Only treat “pops” as aging-related when there is explicit evidence it refers to a grandfather or significantly older parent.
The term “middle-aged” does not count as older adulthood and should not be classified as aging-related.
Tweets about assisted living, nursing homes, or care homes should be classified as aging-related only if an older person is explicitly mentioned in the tweet.
Tweets that refer to older films, old versions of software, “old days,” “old memories,” or anything “old” that is not a person should not be treated as aging-related.

    keyword list: {keywords_text}
    Post: {text}
    """

    # LLM call
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": [{"type": "text", "text": prompt_text.strip()}]}],
        response_format=classification_schema,
        )

    raw = response.choices[0].message.content.strip()
    #print(f"🏷️ LLM output raw: {raw}")

    # Parse JSON safely
    try:
        parsed = json.loads(raw)
        classification = parsed.get("classification", "Not relevant")
    except Exception:
        classification = "Not relevant"

    df.at[i, "classification"] = classification

    print(f"\n📄 Tweet {i+1} of {len(df)}")
    print(f"🏷️ Classification: {classification}")
    #print(f"⏱️ Runtime: {round(time.time() - start_time, 2)} seconds")

# -----------------------------
# Save output
# -----------------------------
df.to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Done. Saved to {OUTPUT_FILE}")


📄 Tweet 1 of 750
🏷️ Classification: Not relevant

📄 Tweet 2 of 750
🏷️ Classification: Not relevant

📄 Tweet 3 of 750
🏷️ Classification: Ageist

📄 Tweet 4 of 750
🏷️ Classification: Not relevant

📄 Tweet 5 of 750
🏷️ Classification: Not relevant

📄 Tweet 6 of 750
🏷️ Classification: Not relevant

📄 Tweet 7 of 750
🏷️ Classification: Not relevant

📄 Tweet 8 of 750
🏷️ Classification: Ageist

📄 Tweet 9 of 750
🏷️ Classification: Not relevant

📄 Tweet 10 of 750
🏷️ Classification: Not relevant

📄 Tweet 11 of 750
🏷️ Classification: Age-positive

📄 Tweet 12 of 750
🏷️ Classification: Ageist

📄 Tweet 13 of 750
🏷️ Classification: Age-positive

📄 Tweet 14 of 750
🏷️ Classification: Not relevant

📄 Tweet 15 of 750
🏷️ Classification: Not relevant

📄 Tweet 16 of 750
🏷️ Classification: Not relevant

📄 Tweet 17 of 750
🏷️ Classification: Not relevant

📄 Tweet 18 of 750
🏷️ Classification: Age-positive

📄 Tweet 19 of 750
🏷️ Classification: Not relevant

📄 Tweet 20 of 750
🏷️ Classification: Not relevant

📄 Twee

In [ ]:
pred_df = pd.read_excel("Gemma3_old_prompt1.xlsx")
ans_df  = pd.read_excel("750-tweets-answer.xlsx")

# Column names
pred_col = "classification"
aging_col = "aging related (yes/ no)"
ageist_col = "ageist (yes/ no)"
pos_col = "age positive (yes/ no)"

# -----------------------------
# Step 1: Create gold label using your rules
# -----------------------------
def create_gold_label(row):
    aging = row[aging_col].strip().upper()
    ageist = row[ageist_col].strip().upper()
    agepos = row[pos_col].strip().upper()

    if aging == "Y" and ageist == "Y":
        return "Ageist"
    elif aging == "Y" and agepos == "Y":
        return "Age-positive"
    else:
        return "Not relevant"

ans_df["gold_label"] = ans_df.apply(create_gold_label, axis=1)

# -----------------------------
# Step 2: Clean predicted label formatting
# -----------------------------
pred_df["pred_label"] = (
    pred_df[pred_col]
    .astype(str)
    .str.strip()
    .str.replace("-", "")       # e.g., "Age-positive" vs "Age positive"
    .str.title()                # normalize casing
)

# Normalize known values
pred_df["pred_label"] = pred_df["pred_label"].replace({
    "Agepositive": "Age-positive",
    "Ageist": "Ageist",
    "Not Relevant": "Not relevant",
})

# -----------------------------
# Step 3: Compare predicted label vs. gold label
# -----------------------------
gold = ans_df["gold_label"].to_numpy()
pred = pred_df["pred_label"].to_numpy()

pred_df["correct"] = pred == gold

accuracy = pred_df["correct"].mean()

print("\n====================================")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct predictions: {pred_df['correct'].sum()} / {len(pred_df)}")
print("====================================\n")

# Save output
pred_df.to_excel("Gemma3_prmp1_accuracy_output.xlsx", index=False)